In [1]:
#!fsharp
#r "nuget: FsLab"

Installed package FsLab version 1.1.6

In [1]:
#!fsharp
open FSharp.Data
open XPlot.Plotly

In [1]:
#!fsharp
let text = CsvFile.Load("C:/Users/bmitc/source/repos/ProgrammingMachineLearning/ProgrammingMachineLearning/data/pizza.txt",
                        separators=",", hasHeaders=true)

let X,Y = [|for row in text.Rows -> (float row.[0], float row.[1])|] |> Array.unzip

In [1]:
#!fsharp
X.[0..4]

index,value
0,13
1,2
2,14
3,23
4,13


In [1]:
#!fsharp
Y.[0..4]

index,value
0,33
1,16
2,32
3,51
4,27


In [1]:
#!fsharp
Scatter(x=X, y=Y, mode = "markers")
|> Chart.Plot
|> Chart.WithXTitle "Reservations"
|> Chart.WithYTitle "Pizzas"

In [1]:
#!fsharp
let predict (x: float[]) weight = Array.map ((*) weight) x

let average (x: float[]) = (Array.fold (+) 0.0 x) / (float x.Length)

let (.+) (x: float[]) (y: float[]) = Array.map2 (+) x y
let (.-) (x: float[]) (y: float[]) = Array.map2 (-) x y
let (.^) (x: float[]) y = Array.map (fun e -> e ** y) x 

let loss (x: float[]) (y: float[]) weight =
    average(((predict x weight) .- y) .^ 2.0)

let train x y iterations learningRate =
    let mutable w = 0.0
    for i in 0..iterations do
        let currentLoss = loss x y w
        //printfn "Iteration %4d => Loss: %.6f" i currentLoss
        if loss x y (w + learningRate) < currentLoss then
            w <- w + learningRate
        elif loss x y (w - learningRate) < currentLoss then
            w <- w - learningRate
    w

In [1]:
#!fsharp
let w = train X Y 10000 0.01
printfn "w=%.3f" w
printfn "Prediction: x=%d => y=%.2f" 20 (predict [|20.0|] w).[0]

w=

1.840

Prediction: x=

20

 => y=

36.80

In [1]:
#!fsharp
let lineX = [| (Array.min X)..1.0..(Array.max X) |]
let lineY = predict lineX w

[Scatter(x=X, y=Y, name="Data", mode = "markers");
 Scatter(x=lineX, y=lineY, name="Prediction")]
|> Chart.Plot
|> Chart.WithXTitle "Reservations"
|> Chart.WithYTitle "Pizzas"

In [1]:
#!fsharp
let predict2 (x: float[]) weight bias = Array.map (fun e -> e*weight + bias) x

let loss2 (x: float[]) (y: float[]) weight bias =
    average(((predict2 x weight bias) .- y) .^ 2.0)

let train2 x y iterations learningRate =
    let mutable weight = 0.0
    let mutable bias = 0.0
    for i in 0..iterations do
        let currentLoss = loss2 x y weight bias
        if loss2 x y (weight + learningRate) bias < currentLoss then
            weight <- weight + learningRate
        elif loss2 x y (weight - learningRate) bias < currentLoss then
            weight <- weight - learningRate
        elif loss2 x y weight (bias + learningRate) < currentLoss then
            bias <- bias + learningRate
        elif loss2 x y weight (bias - learningRate) < currentLoss then
            bias <- bias - learningRate
    weight, bias

In [1]:
#!fsharp
let w, b = train2 X Y 10000 0.01

printfn "w=%.3f, b=%.3f" w b
printfn "Prediction: x=%d => y=%.2f" 20 (predict2 [|20.0|] w b).[0]

w=

1.100

, b=

12.930

Prediction: x=

20

 => y=

34.93

In [1]:
#!fsharp
let lineX = [| (Array.min X)..1.0..(Array.max X) |]
let lineY = predict2 lineX w b

[Scatter(x=X, y=Y, name="Data", mode = "markers");
 Scatter(x=lineX, y=lineY, name="Prediction")]
|> Chart.Plot
|> Chart.WithXTitle "Reservations"
|> Chart.WithYTitle "Pizzas"